In [ ]:
import os
from bs4 import BeautifulSoup
%pip install playwright
!playwright install
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout

In [ ]:
SEASONS = list(range(2016, 2025))

In [ ]:
DATA_DIR = 'games'
STANDINGS_DIR = os.path.join(DATA_DIR, 'standings')
SCORES_DIR = os.path.join(DATA_DIR, 'scores')


# Ensure directories exist
os.makedirs(STANDINGS_DIR, exist_ok=True)
os.makedirs(SCORES_DIR, exist_ok=True)

print(f"Data will be saved in: {DATA_DIR}")
print(f"Standings will be saved in: {STANDINGS_DIR}")
print(f"Scores will be saved in: {SCORES_DIR}")

In [ ]:
import time

async def get_html(url, selector, sleep=30, retries=5):
    html = None
    for _ in range(1, retries+1):
        time.sleep(sleep)
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f'Timeout on attempt {_} for {url}')
            continue
        else:
            break
    return html

In [ ]:
async def scrape_seasons(season):
    url = f'https://www.basketball-reference.com/leagues/NBA_{season}_games.html'
    html = await get_html(url, '#content .filter')
    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    href = [link['href'] for link in links]
    standings_pages = [f"https://www.basketball-reference.com{h}" for h in href] 
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            print(f'{save_path} already exists')
            continue

        html = await get_html(url, '#all_schedule')
        with open(save_path, 'w+') as f:
            f.write(html)


In [ ]:
season = 2010
url = f'https://www.basketball-reference.com/leagues/NBA_{season}_games.html'

In [ ]:
standings_files = os.listdir(STANDINGS_DIR)

In [ ]:
async def scrape_box_scores(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    href = [link.get("href") for link in links]
    box_scores = [h for h in href if h and 'boxscores' in h and '.html' in h]
    box_scores = [f"https://www.basketball-reference.com{h}" for h in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            print(f'{save_path} already exists')
            continue

        html = await get_html(url, '#content')
        if not html:
            print(f'Failed to get {url}')
            continue
        with open(save_path, 'w+') as f:
            f.write(html)

In [ ]:
standings_files = [s for s in standings_files if '.html' in s]

for file in standings_files:
    file_path = os.path.join(STANDINGS_DIR, file)
    await scrape_box_scores(file_path)

In [ ]:
standings_files
